### Imports and Azure OpenAI client setup

In [1]:
import os
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt


deployment = "ai-roadshow" #gpt-3.5-turbo-0613
client = AzureOpenAI(
    api_key=os.getenv("AI_ROADSHOW_AOAI_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint=os.getenv("AI_ROADSHOW_AOIA_ENDPOINT")
)

print("api_key:", os.getenv("AI_ROADSHOW_AOAI_KEY"))
print("api_version:", "2023-12-01-preview")
print("azure_endpoint:", os.getenv("AI_ROADSHOW_AOIA_ENDPOINT"))


api_key: 4db5f459c5fa4479b209017442ee7c1b
api_version: 2023-12-01-preview
azure_endpoint: https://oai-wth.openai.azure.com/


In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=deployment):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e



In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city for which to get the weather",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location the user wants to know its weather",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [4]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Cómo estará el clima hoy en Bogotá?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dhqSs1TlbeN2J3wjX9DrhqaC', function=Function(arguments='{\n  "location": "Bogotá",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])

In [5]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "I am from Seattle and prefer Fahrenhgeit but I'm visiting Bogotá. What will be the weather today?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_60UleHE8LxBHYB4u4zqzI6fo', function=Function(arguments='{\n  "location": "Bogotá",\n  "format": "fahrenheit"\n}', name='get_current_weather'), type='function')])

In [6]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Cómo estará el clima?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content='En qué ciudad te gustaría saber el clima?', role='assistant', function_call=None, tool_calls=None)

In [7]:
messages.append({"role": "user", "content": "Estoy en Medellín"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uT8LOM8Ib3XllHzkWDKkwxom', function=Function(arguments='{\n  "location": "Medellín",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')])

In [8]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Como estará el clima en Cartagena por los siguientes x días?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content='¿Podrías proporcionar la cantidad de días para la previsión del clima en Cartagena?', role='assistant', function_call=None, tool_calls=None)

In [9]:
messages.append({"role": "user", "content": "una semana"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_R7BExwGNK2rBrz7DGmR2eVLc', function=Function(arguments='{\n  "location": "Cartagena",\n  "format": "celsius",\n  "num_days": 7\n}', name='get_n_day_weather_forecast'), type='function')]), content_filter_results={})

In [10]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "How will be the weather in Cartagena for the next x days?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content='Sure, I can help you with that. Could you please specify how many days ahead you would like to know the weather forecast for?', role='assistant', function_call=None, tool_calls=None)

In [11]:
messages.append({"role": "user", "content": "A week please"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='Thank you for providing the number of days. Let me fetch the weather forecast for Cartagena for the next 7 days.', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ToBCxBKOfRTRSKzdyITVvyyY', function=Function(arguments='{\n  "location": "Cartagena",\n  "format": "celsius",\n  "num_days": 7\n}', name='get_n_day_weather_forecast'), type='function')]), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})